# 🧠 ClassMind — AI Study Assistant (DE/EN)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/<YOUR_USERNAME>/<YOUR_REPO>/blob/main/AI_Study_Assistant_fixed.ipynb)

**How to use:** Runtime → **Run all**, then upload a PDF/PPTX when prompted. Outputs are saved to `/outputs`:
`deck_summary.txt`, `slide_summaries.json`, `mindmap.mmd`, `flashcards.csv`.


In [1]:
#  Setup
!pip -q install pymupdf python-pptx pillow transformers sentence-transformers scikit-learn tqdm pytesseract pandas
!sudo apt-get -qq update && sudo apt-get -qq install -y tesseract-ocr tesseract-ocr-deu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 10.1 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tesseract-ocr-deu.
(Reading database ... 126675 files and directories currently i

In [2]:
import os, io, re, json, csv, string
from collections import Counter, defaultdict
from tqdm import tqdm
import fitz  # PyMuPDF
from pptx import Presentation
from PIL import Image
import numpy as np
import pandas as pd

from transformers import pipeline
from sentence_transformers import SentenceTransformer

# Models (multilingual; CPU-safe summarizer)
summarizer = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum", device=-1)
embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

os.makedirs("outputs", exist_ok=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
from google.colab import files
up = files.upload()   # pick your PDF or PPTX
file_path = list(up.keys())[0]
print("Uploaded:", file_path)


Saving Md Touhidur Rahman_Report.docx.pdf to Md Touhidur Rahman_Report.docx.pdf
Uploaded: Md Touhidur Rahman_Report.docx.pdf


In [4]:
import pytesseract
OCR_LANG = "eng+deu"; OCR_PSM = "6"; OCR_OEM = "3"
OCR_CONFIG = f"--oem {OCR_OEM} --psm {OCR_PSM}"

def _clean_text(s):
    s = re.sub(r"[ \t]+"," ", s or ""); s = re.sub(r"\n{3,}","\n\n", s); return s.strip()
def _ocr_pil(img): return _clean_text(pytesseract.image_to_string(img, lang=OCR_LANG, config=OCR_CONFIG))
def _pdf_page_to_image(page, dpi=300):
    pix = page.get_pixmap(dpi=dpi)
    return Image.open(io.BytesIO(pix.tobytes("png"))).convert("RGB")

def extract_pdf_text(path, ocr=True, min_chars_no_ocr=25, dpi=300):
    doc = fitz.open(path); out=[]; ocrn=0
    for i, p in enumerate(doc):
        t = (p.get_text() or "").strip(); used=False; img_only=False
        if (not t or len(t)<min_chars_no_ocr) and ocr:
            img = _pdf_page_to_image(p, dpi=dpi); t=_ocr_pil(img); used=True; img_only=True; ocrn+=1
        out.append({"index":i,"text":_clean_text(t),"source":"pdf","ocr_used":used,"image_only":img_only})
    print(f"OCR used on {ocrn}/{len(out)} PDF pages."); return out

def extract_pptx_text(path, ocr=True):
    prs = Presentation(path); out=[]; ocrn=0
    for i, s in enumerate(prs.slides):
        texts=[]; images=[]
        for sh in s.shapes:
            if hasattr(sh,"text") and sh.text: texts.append(sh.text)
            if getattr(sh,"shape_type", None)==13 and ocr:  # PICTURE
                try: images.append(_ocr_pil(Image.open(io.BytesIO(sh.image.blob)).convert("RGB")))
                except: pass
        raw=_clean_text("\n".join(texts)); used=False; img_only=False
        if ocr and (not raw or len(raw)<25) and images:
            ocr_txt=_clean_text("\n".join([t for t in images if t]));
            if ocr_txt: raw=(raw+"\n"+ocr_txt).strip() if raw else ocr_txt; used=True; img_only=(len(_clean_text("\n".join(texts)))<5); ocrn+=1
        out.append({"index":i,"text":raw,"source":"pptx","ocr_used":used,"image_only":img_only})
    print(f"OCR used on {ocrn}/{len(out)} PPTX slides."); return out

slides = extract_pdf_text(file_path, ocr=True) if file_path.lower().endswith(".pdf") else extract_pptx_text(file_path, ocr=True)
print("Slides:", len(slides))


OCR used on 0/7 PDF pages.
Slides: 7


In [5]:
def summarize_text(txt):
    txt = (txt or "").strip()
    if not txt or len(txt.split()) < 25: return txt
    return summarizer(txt, max_length=160, min_length=60, do_sample=False)[0]["summary_text"].strip()

slide_summaries = []
for s in tqdm(slides):
    slide_summaries.append({**s, "summary": summarize_text(s["text"])})

with open("outputs/slide_summaries.json","w",encoding="utf-8") as f:
    json.dump(slide_summaries, f, ensure_ascii=False, indent=2)

slide_summaries[0]


100%|██████████| 7/7 [04:56<00:00, 42.31s/it]


{'index': 0,
 'text': 'Spatio-Temporal Techniques for Image Enhancement \nin Medical Imaging \n \nMd Touhidur Rahman \nDepartment of Data Science \nFriedrich-Alexander University Erlangen-Nuremberg, \nBayern, Germany \ntouhidur.rahman@fau.de \n \n \nAbstract—In the field of medical diagnostics, sharp and \naccurate images are crucial for making the right decisions. \nMagnetic Resonance Imaging (MRI) is one of the most widely \nused imaging methods in the world but sometimes it struggles \nwith a critical tradeoff: achieving high spatial resolution while \nthe scan times are short and with the least amount of artefacts \ncaused by motion. Spatio-temporal enhancement techniques try \nto deal with this property by using the information about \nthe\u2002image itself (spatial) and of its behaviour along time \n(temporal). This report provides an in-depth review of three \nrecently published research papers of relevance that apply\u2002deep \nlearning in medical images enhancement using spat

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

pages = [s.get("text","") for s in slides if s.get("text")]
doc = "\n".join(pages)
sentences = [t.strip() for t in re.split(r'(?<=[.!?])\s+', doc) if len(t.strip().split())>=6]

# filters to avoid captions/urls/watermarks
def keep_sentence(s):
    if sum(c.isalpha() for c in s)/max(1,len(s)) < 0.5: return False
    if re.search(r'(Chair of|Page \d+|\*{3}Not for sharing\*{3}|https?://|Wikimedia|CC BY|Pixabay)', s, re.I): return False
    return True
sentences = [s for s in sentences if keep_sentence(s)]

stop = """der die das ein eine und oder für von mit ohne zu auf an im in aus ist sind war waren sein auch sowie
the a an and or for from with without into in on of to is are was were be been being this that these those it its by as at if then else when while not
data using use based results method methods model models paper figure table slide page""".split()
vec = TfidfVectorizer(stop_words=stop, max_df=0.9, min_df=2, ngram_range=(1,2))
X = vec.fit_transform(sentences); scores = X.sum(axis=1).A.ravel()

def mmr_select(sentences, scores, X, max_words=260, lambda_div=0.7):
    chosen_idx=[]; remaining=set(range(len(sentences))); sim=cosine_similarity(X, X)
    i0=int(scores.argmax()); chosen_idx.append(i0); remaining.remove(i0); total=len(sentences[i0].split())
    while remaining and total < max_words:
        best_i, best_v = None, -1e9
        for i in remaining:
            rel=scores[i]; div=0 if not chosen_idx else max(sim[i, chosen_idx])
            val=lambda_div*rel - (1-lambda_div)*div
            if val>best_v and total+len(sentences[i].split())<=max_words+40:
                best_v, best_i = val, i
        if best_i is None: break
        chosen_idx.append(best_i); remaining.remove(best_i); total+=len(sentences[best_i].split())
    return " ".join(sentences[i] for i in sorted(chosen_idx))

deck_summary = mmr_select(sentences, scores, X, max_words=260, lambda_div=0.7)
deck_summary = re.sub(r'\s+',' ', deck_summary).strip()
with open("outputs/deck_summary.txt","w",encoding="utf-8") as f: f.write(deck_summary)
print(deck_summary[:500])


Magnetic Resonance Imaging (MRI) is one of the most widely used imaging methods in the world but sometimes it struggles with a critical tradeoff: achieving high spatial resolution while the scan times are short and with the least amount of artefacts caused by motion. Keywords—Medical imaging, deep learning, spatio-temporal data, MRI enhancement, U-Net, super-resolution, motion artefact reduction, dynamic MRI, real-time image processing I.​ INTRODUCTION Medical imaging has become a vital tool now


In [7]:
corpus = [f"Slide {s['index']}: {s['summary'] or s['text']}" for s in slide_summaries]
embs = embedder.encode(corpus, convert_to_numpy=True, normalize_embeddings=True)

def search(query, top_k=5):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    sims = (embs @ q.T).ravel()
    idx = sims.argsort()[::-1][:top_k]
    return [{"slide_index": slide_summaries[i]["index"],
             "score": float(sims[i]),
             "preview": (slide_summaries[i]["summary"] or slide_summaries[i]["text"])[:240]} for i in idx]

search("Markow-Eigenschaft")


[{'slide_index': 0,
  'score': 0.24245168268680573,
  'preview': 'In our series of letters from African journalists, Dr Touhidur Rahman looks at how deep learning is being applied to improve medical imaging in real world medical scenarios. Here, he explains how spatio-temporal enhancement techniques are a'},
 {'slide_index': 4,
  'score': 0.2267385721206665,
  'preview': 'In our series of letters from across the world, Dr John Wright looks at how spatio-temporal learning (DDoS-UNet) could improve the MRI images of patients. Here, he explains how it works and what it does.'},
 {'slide_index': 2,
  'score': 0.19386336207389832,
  'preview': 'In our series of letters from across the world, scientists have developed a deep learning model to improve the spatio-temporal resolution of brain MRI scans. This is the first paper to be published in the Journal of Ultrasound Medicine (DDo'},
 {'slide_index': 3,
  'score': 0.16172504425048828,
  'preview': "Scientists have developed a deep-learning 

In [8]:
def top_keywords(text, k=3):
    words = re.findall(r"[A-Za-zÄÖÜäöüß][A-Za-zÄÖÜäöüß\-]{2,}", text or "")
    stop = set("""der die das ein eine und oder für von mit ohne zu auf an im in aus ist sind war waren sein auch sowie
the a an and or for from with without into in on of to is are was were be been being this that these those it its by as at if then else when while not""".split())
    words = [w.lower() for w in words if w.lower() not in stop]
    return [w for w,_ in Counter(words).most_common(k)]

kw_to_slides = defaultdict(list)
for s in slide_summaries:
    base = s["summary"] or s["text"]
    for kw in top_keywords(base, 3): kw_to_slides[kw].append(s["index"])

top_kws = list(kw_to_slides.keys())[:12]
title = os.path.basename(file_path)
lines = ["mindmap", f"  root(({title}))"]
for kw in top_kws:
    lines.append(f"    {kw}({kw})")
    for idx in sorted(kw_to_slides[kw])[:6]:
        lines.append(f"      s{idx}([Slide {idx}])")

with open("outputs/mindmap.mmd","w",encoding="utf-8") as f: f.write("\n".join(lines))
print("mindmap.mmd written (paste into https://mermaid.live)")


mindmap.mmd written (paste into https://mermaid.live)


In [9]:
def split_points(text):
    bullets = re.findall(r"(?:^|\n)[\-\•\*]\s*(.+)", text or "")
    if bullets: return [b.strip() for b in bullets if len(b.strip())>3][:4]
    sents = re.split(r"(?<=[.!?])\s+", text or "")
    return [s for s in sents if len(s.split())>=6][:3]

def detect_de(text):
    t=(text or "").lower()
    if any(c in t for c in "äöüÄÖÜß"): return True
    de_hits=sum(w in t for w in ["und","der","die","das","ist","nicht","ein","eine","oder","auch"])
    en_hits=sum(w in t for w in ["and","the","is","are","not","or","also","a","an"])
    return de_hits>=en_hits

rows=[]
for s in slide_summaries:
    base=s["summary"] or s["text"]
    if not base: continue
    de=detect_de(base); items=split_points(base)
    if items:
        for j,it in enumerate(items,1):
            q = f"Nenne einen Kerngedanken (Folie {s['index']}, Punkt {j}):" if de else f"Name one key idea (slide {s['index']}, point {j}):"
            rows.append([q, it, f"ClassMind;{'DE' if de else 'EN'}"])
    else:
        q = f"Was ist die Kernaussage von Folie {s['index']}?" if de else f"What is the main idea of slide {s['index']}?"
        rows.append([q, base, f"ClassMind;{'DE' if de else 'EN'}"])

with open("outputs/flashcards.csv","w",encoding="utf-8",newline="") as f:
    csv.writer(f).writerow(["Question","Answer","Tags"])
with open("outputs/flashcards.csv","a",encoding="utf-8",newline="") as f:
    w=csv.writer(f); w.writerows(rows)

print(f"flashcards.csv written — {len(rows)} cards")


flashcards.csv written — 14 cards


In [10]:
from google.colab import files
for p in ["outputs/deck_summary.txt","outputs/slide_summaries.json","outputs/mindmap.mmd","outputs/flashcards.csv"]:
    try: files.download(p)
    except Exception as e: print("Skip:", p, "->", e)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Upload the .ipynb you want to publish, clean widget metadata, and save a fixed copy.
!pip -q install nbformat==5.10.4
import nbformat as nbf
from google.colab import files

print("Pick the .ipynb you want to publish to GitHub:")
up = files.upload()
name = list(up.keys())[0]

nb = nbf.reads(up[name].decode("utf-8"), as_version=4)

# Remove repo-level widget metadata
nb.metadata.pop("widgets", None)
# Ensure basic metadata (helps some renderers)
nb.metadata.setdefault("kernelspec", {"name":"python3","display_name":"Python 3"})
nb.metadata.setdefault("language_info", {"name":"python"})

# Remove cell-level widget metadata and ensure outputs fields are present
for c in nb.cells:
    if isinstance(c.get("metadata", {}), dict):
        c.metadata.pop("widgets", None)
    if c.cell_type == "code":
        c.outputs = c.get("outputs", []) or []   # must exist, even if empty
        if "execution_count" not in c:
            c.execution_count = None

# Write cleaned notebook
fixed = name.rsplit(".ipynb", 1)[0] + "_fixed.ipynb"
with open(fixed, "w", encoding="utf-8") as f:
    nbf.write(nb, f)

files.download(fixed)
print("Cleaned file ready:", fixed)


Pick the .ipynb you want to publish to GitHub:
